## Alpha Vantage - PE Ratio Example

In [4]:
import os # for referencing environment variables
from dotenv import load_dotenv # for loading environment variables 
import requests

In [5]:
load_dotenv() # load environment variables from .env file
alpha_vantage_api_key = os.getenv('ALPHA_VANTAGE_API_KEY') # Get your API key from https://www.alphavantage.co/support/#api-key

In [6]:
ticker = 'AAPL'
url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={alpha_vantage_api_key}'
r = requests.get(url)
data = r.json()
pe_ratio = float(data['PERatio'])
pe_ratio

35.85

### Reducer function scratchwork

In [1]:
def flatten_to_single_string_in_list_reducer(*args):
    '''
        A reducer function that returns a single string value in a list.
        Used for cases where a single values is needed in the state 
        for a given key when merging the state from two sub-graphs.
        NOTE: There appears to be no way for langgraph to natively 
        merge state for a key of type string between two sub-graphs. 
        This behavior appears to be different than when merging the state 
        from two different nodes which are not sub-graphs.
    '''
    def flatten(input_data):
        for item in input_data:
            if isinstance(item, (list, tuple)):
                yield from flatten(item)
            else:
                yield item
    
    # Flatten all arguments into a single list
    flat_list = list(flatten(args))
    
    # Find the first non-null string in the flattened list
    for item in flat_list:
        if item is not None and isinstance(item, str):
            return [item]
    
    return []

In [2]:
print(flatten_to_single_string_in_list_reducer('AAPL'))  # Output: ['AAPL']
print(flatten_to_single_string_in_list_reducer(['AAPL', 'AAPL']))  # Output: ['AAPL']
print(flatten_to_single_string_in_list_reducer(['AAPL', 'AAPL'], ['AAPL']))  # Output: ['AAPL']
print(flatten_to_single_string_in_list_reducer(None, ['AAPL'], ['AAPL']))  # Output: ['AAPL']
print(flatten_to_single_string_in_list_reducer(None, 'AAPL'))  # Output: ['AAPL']
print(flatten_to_single_string_in_list_reducer(['AAPL', None, 'BP']))  # Output: ['AAPL']
print(flatten_to_single_string_in_list_reducer(None, None, 'AAPL', None, ['BP']))  # Output: ['AAPL']
print(flatten_to_single_string_in_list_reducer([['AAPL'], ['AAPL']]))  # Output: ['AAPL']
print(flatten_to_single_string_in_list_reducer((['AAPL', 'AAPL'], ['AAPL'])))  # Output: ['AAPL']
print(flatten_to_single_string_in_list_reducer((None, 'AAPL'), (None, 'BP')))  # Output: ['AAPL']
print(flatten_to_single_string_in_list_reducer([['AAPL'], ['AAPL']]))

['AAPL']
['AAPL']
['AAPL']
['AAPL']
['AAPL']
['AAPL']
['AAPL']
['AAPL']
['AAPL']
['AAPL']
['AAPL']
